In [3]:
def convert_to_square(image, new_size=None):
    square_size = np.max(image.shape)
    h, w = image.shape[0], image.shape[1]
    delta_w = square_size - w
    delta_h = square_size - h
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    square_image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_REPLICATE)
    if not new_size is None:
        square_image = cv2.resize(square_image, dsize=(new_size, new_size), interpolation=cv2.INTER_CUBIC)
    return square_image